# BingSearch using Kernel 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documentation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview).


Prepare a Semantic Kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.6.3"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.6.3-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.6.3-alpha"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.TemplateEngine;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.6.3 Microsoft.SemanticKernel.Plugins.Core, 1.6.3-alpha Microsoft.SemanticKernel.Plugins.Web, 1.6.3-alpha

Add the following namespaces to use the Bing Search connector.

In [2]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

Enter your Bing Search Key in BING_KEY using `InteractiveKernel` method as introduced in [`.NET Interactive`](https://github.com/dotnet/interactive/blob/main/docs/kernels-overview.md)

In [3]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string BING_KEY = (await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search Key")).GetClearTextPassword();

Below are some examples on how [`WebSearchEnginePlugin`](../src/Plugins/Plugins.Web/WebSearchEnginePlugin.cs) can be used in SK. 

In [5]:
private static async Task Example1Async(Microsoft.SemanticKernel.Kernel kernel)
{
        Console.WriteLine("Example 1");

        // Run
        var question = "What is quantum tunnelling";
        var function = kernel.Plugins["bing"]["search"];
        var bingResult = await kernel.InvokeAsync(function, new() { ["query"] = question });

        Console.WriteLine(question);
        Console.WriteLine("----");
        Console.WriteLine(bingResult);
        Console.WriteLine();

        /* OUTPUT:

            What is quantum tunnelling
            ----
            In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical
            phenomenon in which an object such as an electron or atom passes through a potential energy
            barrier that, according to classical mechanics, the object does not have sufficient energy to
            enter or surmount.

       */
}

In [6]:
private static async Task Example2Async(Microsoft.SemanticKernel.Kernel kernel)
{
    Console.WriteLine("Example 2");

    //The following function only works in interactive notebooks
    string question = await InteractiveKernel.GetInputAsync("Please ask your question");

    var function = kernel.Plugins["bing"]["search"];
    var bingResult = await kernel.InvokeAsync(function, new() { ["query"] = question });

    Console.WriteLine(bingResult);
}

In [7]:
#pragma warning disable SKEXP0050

// Load Bing plugin
var bingConnector = new BingConnector(BING_KEY);

kernel.ImportPluginFromObject(new WebSearchEnginePlugin(bingConnector), "bing");

await Example1Async(kernel);
await Example2Async(kernel);

Example 1
What is quantum tunnelling
----
["Quantum tunnelling increases the probability of penetrating this barrier. Though this probability is still low, the extremely large number of nuclei in the core of a star is sufficient to sustain a steady fusion reaction. Radioactive decay. Radioactive decay is the process of emission of particles and energy from the unstable nucleus of an atom ...","Quantum tunneling is a phenomenon where an electron is able to phase through a barrier and move to the other side. It is a quantum phenomenon that occurs when particles move through a barrier that, according to the theories of classical physics, should be impossible to pass through. When an object encounters a barrier, it is an intuitive ...","Quantum tunneling is a phenomenon in which a particle can cross a potential barrier that is higher than its energy. This chapter explains the principles and applications of quantum tunneling, using the Schrödinger equation and the wave function. Learn more 